In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
import csv
import re

In [2]:
df = pd.read_csv("trainingLabels.csv")
df.head()

,ID,d1dlwa_,d1dlya_,d1idra_,d1scta_,d1sctb_,d3sdha_,d1b0b__,d1h97a_,d1vrea_,...,d1qbha_,d1f9xa_,d1g73c_,d1g73d_,d1i3oe_,d1jd5a_,d1e31a_,d1f81a_,d1hc7a3,d1exka_
0,d1dlwa_,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,d1dlya_,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,d1idra_,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,d1scta_,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,d1sctb_,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
proteins = df.loc[:,'ID']

In [4]:
out_directory = "feature_matricies/"
hmmer_directory = "parsed_hmmer/"
psi_blast_directory = "psi_blast_files/"
num_proteins = len(proteins)

In [7]:
def create_feature_matrix(protein):
    outfile = open(out_directory+protein+'.csv', 'w')
    hmmer_files = glob.glob(hmmer_directory + '*' + protein + ".csv")
    psi_blast_files = glob.glob(psi_blast_directory + '*' + protein)
    writer_fields = ['psi_bitscore', 'psi_eval', 'hmmer_eval', 'psi_rank', 'hmmer_rank']
    try:
        hmmer = pd.read_csv(hmmer_files[0])
        psi_header = ['query','target','eval','bitscore']
        psi_blast = pd.read_csv(psi_blast_files[0], names = psi_header)
    except:
        return 
    #hmmer = hmmer.set_index('ID')
    #psi_blast = psi_blast.set_index('target')
    #print(psi_blast['target'] == 'd1lla_2')(1).any()
    #print(psi_blast.at['d1lla_2', 'bitscore'])
    #print(psi_blast.loc[psi_blast['target'] == 'd1lla_2', 'bitscore'].item())

    names = []
    psi_bitscore = []
    psi_eval = []
    psi_rank = []
    hmmer_eval = []
    hmmer_rank = []
    for target in proteins:

        try:
            new_psi_bitscore = psi_blast.loc[psi_blast['target'] == target,'bitscore'].item()
            new_psi_eval = psi_blast.loc[psi_blast['target'] == target,'eval'].item()
            new_psi_rank = 1/(psi_blast.loc[psi_blast['target'] == target].index[0].item()+1)

        except:
            new_psi_bitscore = np.NaN
            new_psi_eval = np.NaN
            new_psi_rank = 1/num_proteins
        try:
            new_hmmer_eval = hmmer.loc[hmmer['ID'] == target,'Eval'].item()
            new_hmmer_rank = 1/(hmmer.loc[hmmer['ID'] == target, 'rank'].item())

        except:
            new_hmmer_eval = np.NaN
            new_hmmer_rank = 1/num_proteins
        names.append(target)
        psi_bitscore.append(new_psi_bitscore)
        psi_eval.append(new_psi_eval)
        psi_rank.append(new_psi_rank)
        hmmer_eval.append(new_hmmer_eval)
        hmmer_rank.append(new_hmmer_rank)
    '''print(len(psi_bitscore))
    print(len(psi_eval))
    print(len(psi_rank))
    print(len(hmmer_eval))
    print(len(hmmer_rank))
    print(psi_bitscore)'''
    frameDict = {'ID': target,
                 'psi_bitscore':psi_bitscore,
                 'psi_eval': psi_eval,
                'psi_rank': psi_rank,
                'hmmer_eval': hmmer_eval,
                'hmmer_rank': hmmer_rank}
    #print(frameDict)
    ff = pd.DataFrame(frameDict)
    
    psi_bitscore_max = ff['psi_bitscore'].max()
    ff['psi_bitscore'] = ff['psi_bitscore'].apply(lambda x: x/psi_bitscore_max)
    ff['psi_bitscore'] = ff['psi_bitscore'].fillna(0)
    
    psi_eval_max = ff['psi_eval'].max()
    ff['psi_eval'] = ff['psi_eval'].apply(lambda x: x/psi_eval_max)
    ff['psi_eval'] = ff['psi_eval'].fillna(1)
    
    hmmer_eval_max = ff['hmmer_eval'].max()
    ff['hmmer_eval'] = ff['hmmer_eval'].apply(lambda x: x/hmmer_eval_max)
    ff['hmmer_eval'] = ff['hmmer_eval'].fillna(1)
    
    #print(ff)
    ff.to_csv(outfile,header=True, index=True)
    

In [15]:
from multiprocessing import Pool

In [13]:
proteinlist = proteins.tolist()

In [18]:
def imap_unordered_bar(func, args, n_processes = 4):
    p = Pool(n_processes)
    res_list = []
    with tqdm(total = len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

In [16]:
with Pool(processes=4) as p:
        max_ = num_proteins
        with tqdm(total=max_) as pbar:
            for i, _ in tqdm(enumerate(p.imap_unordered(create_feature_matrix, proteinlist))):
                pbar.update()






  0%|          | 0/7320 [00:00<?, ?it/s]

0it [00:00, ?it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/myPC/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/myPC/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/myPC/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  0%|          | 10/7320 [00:34<8:58:00,  4.42s/it]
10it [00:34,  3.41s/it]


KeyboardInterrupt: 

In [19]:
imap_unordered_bar(create_feature_matrix, proteinlist)


  0%|          | 0/7320 [00:00<?, ?it/s]

0it [00:00, ?it/s]
  0%|          | 1/7320 [00:10<22:13:32, 10.93s/it]

1it [00:10, 10.89s/it]
  0%|          | 2/7320 [00:11<15:37:01,  7.68s/it]

2it [00:10,  5.50s/it]
  0%|          | 3/7320 [00:11<11:10:52,  5.50s/it]

3it [00:11,  3.80s/it]
  0%|          | 4/7320 [00:22<14:20:04,  7.05s/it]

4it [00:22,  5.52s/it]
  0%|          | 5/7320 [00:22<10:12:46,  5.03s/it]

5it [00:22,  4.48s/it]
  0%|          | 6/7320 [00:22<7:23:13,  3.64s/it] 

6it [00:22,  3.79s/it]
  0%|          | 7/7320 [00:25<6:38:41,  3.27s/it]

7it [00:25,  3.60s/it]
  0%|          | 8/7320 [00:34<10:05:42,  4.97s/it]

8it [00:34,  4.27s/it]
  0%|          | 9/7320 [00:34<7:18:16,  3.60s/it] 

9it [00:34,  3.84s/it]
  0%|          | 10/7320 [00:34<5:20:15,  2.63s/it]

10it [00:34,  3.49s/it]
  0%|          | 11/7320 [00:50<13:00:54,  6.41s/it]

11it [00:50,  4.56s/it]
  0%|          | 12/7320 [01:00<15:07:19,  7.45s/it]

12it [00:59,  5.00s/it]
  0%|          | 13/73

  1%|▏         | 96/7320 [05:58<5:16:18,  2.63s/it]

96it [05:57,  3.73s/it]
  1%|▏         | 97/7320 [06:04<7:34:35,  3.78s/it]

97it [06:04,  3.76s/it]
  1%|▏         | 98/7320 [06:05<5:43:10,  2.85s/it]

98it [06:05,  3.73s/it]
  1%|▏         | 99/7320 [06:05<4:08:30,  2.06s/it]

99it [06:05,  3.69s/it]
  1%|▏         | 100/7320 [06:09<5:12:33,  2.60s/it]

100it [06:09,  3.69s/it]
  1%|▏         | 101/7320 [06:15<7:40:49,  3.83s/it]

101it [06:15,  3.72s/it]
  1%|▏         | 102/7320 [06:16<5:47:45,  2.89s/it]

102it [06:16,  3.69s/it]
  1%|▏         | 103/7320 [06:16<4:15:50,  2.13s/it]

103it [06:16,  3.66s/it]
  1%|▏         | 104/7320 [06:21<5:27:13,  2.72s/it]

104it [06:21,  3.66s/it]
  1%|▏         | 105/7320 [06:27<7:50:34,  3.91s/it]

105it [06:27,  3.69s/it]
  1%|▏         | 106/7320 [06:28<5:47:30,  2.89s/it]

106it [06:28,  3.66s/it]
  1%|▏         | 107/7320 [06:32<6:23:02,  3.19s/it]

107it [06:32,  3.66s/it]
  1%|▏         | 108/7320 [06:38<8:32:49,  4.27s/it]

108it 

  3%|▎         | 189/7320 [10:51<4:46:42,  2.41s/it]

189it [10:51,  3.45s/it]
  3%|▎         | 190/7320 [10:58<7:36:16,  3.84s/it]

190it [10:58,  3.47s/it]
  3%|▎         | 191/7320 [11:01<6:59:19,  3.53s/it]

191it [11:01,  3.46s/it]
  3%|▎         | 192/7320 [11:02<5:36:42,  2.83s/it]

192it [11:02,  3.45s/it]
  3%|▎         | 193/7320 [11:09<7:49:29,  3.95s/it]

193it [11:09,  3.47s/it]
  3%|▎         | 194/7320 [11:09<5:50:25,  2.95s/it]

194it [11:09,  3.45s/it]
  3%|▎         | 195/7320 [11:12<5:45:09,  2.91s/it]

195it [11:12,  3.45s/it]
  3%|▎         | 196/7320 [11:13<4:44:00,  2.39s/it]

196it [11:13,  3.44s/it]
  3%|▎         | 197/7320 [11:21<7:42:42,  3.90s/it]

197it [11:21,  3.46s/it]
  3%|▎         | 198/7320 [11:21<5:49:05,  2.94s/it]

198it [11:21,  3.44s/it]
  3%|▎         | 199/7320 [11:24<5:43:42,  2.90s/it]

199it [11:24,  3.44s/it]
  3%|▎         | 200/7320 [11:25<4:29:40,  2.27s/it]

200it [11:25,  3.43s/it]
  3%|▎         | 201/7320 [11:32<7:18:27,  3.70s/it]

  4%|▍         | 284/7320 [15:48<5:17:47,  2.71s/it]

284it [15:48,  3.34s/it]
  4%|▍         | 285/7320 [15:53<6:40:48,  3.42s/it]

285it [15:53,  3.34s/it]
  4%|▍         | 286/7320 [15:55<6:08:00,  3.14s/it]

286it [15:55,  3.34s/it]
  4%|▍         | 287/7320 [16:04<9:17:02,  4.75s/it]

287it [16:04,  3.36s/it]
  4%|▍         | 288/7320 [16:06<7:55:08,  4.05s/it]

288it [16:06,  3.36s/it]
  4%|▍         | 289/7320 [16:08<6:29:28,  3.32s/it]

289it [16:08,  3.35s/it]
  4%|▍         | 290/7320 [16:10<5:40:24,  2.91s/it]

290it [16:10,  3.35s/it]
  4%|▍         | 291/7320 [16:15<7:03:47,  3.62s/it]

291it [16:15,  3.35s/it]
  4%|▍         | 292/7320 [16:20<7:41:55,  3.94s/it]

292it [16:20,  3.36s/it]
  4%|▍         | 293/7320 [16:22<6:29:42,  3.33s/it]

293it [16:22,  3.35s/it]
  4%|▍         | 294/7320 [16:27<7:31:45,  3.86s/it]

294it [16:27,  3.36s/it]
  4%|▍         | 295/7320 [16:30<7:08:03,  3.66s/it]

295it [16:30,  3.36s/it]
  4%|▍         | 296/7320 [16:31<5:36:10,  2.87s/it]

377it [21:31,  3.43s/it]
  5%|▌         | 378/7320 [21:41<11:12:39,  5.81s/it]

378it [21:41,  3.44s/it]
  5%|▌         | 379/7320 [21:41<7:57:09,  4.12s/it] 

379it [21:41,  3.43s/it]
  5%|▌         | 380/7320 [21:45<7:58:33,  4.14s/it]

380it [21:45,  3.44s/it]
  5%|▌         | 381/7320 [21:52<9:15:33,  4.80s/it]

381it [21:52,  3.44s/it]
  5%|▌         | 382/7320 [21:55<8:09:33,  4.23s/it]

382it [21:55,  3.44s/it]
  5%|▌         | 383/7320 [21:55<5:54:37,  3.07s/it]

383it [21:55,  3.43s/it]
  5%|▌         | 384/7320 [21:59<6:37:00,  3.43s/it]

384it [21:59,  3.44s/it]
  5%|▌         | 385/7320 [22:07<9:14:42,  4.80s/it]

385it [22:07,  3.45s/it]
  5%|▌         | 386/7320 [22:09<7:36:00,  3.95s/it]

386it [22:09,  3.44s/it]
  5%|▌         | 387/7320 [22:10<5:53:10,  3.06s/it]

387it [22:10,  3.44s/it]
  5%|▌         | 388/7320 [22:14<6:17:05,  3.26s/it]

388it [22:14,  3.44s/it]
  5%|▌         | 389/7320 [22:22<9:00:59,  4.68s/it]

389it [22:22,  3.45s/it]
  5%|▌         | 390/7320

...

 95%|█████████▌| 6975/7320 [8:11:13<19:37,  3.41s/it]

6975it [8:11:13,  4.23s/it]
 95%|█████████▌| 6976/7320 [8:11:13<14:04,  2.46s/it]

6976it [8:11:13,  4.22s/it]
 95%|█████████▌| 6977/7320 [8:11:16<14:48,  2.59s/it]

6977it [8:11:16,  4.22s/it]
 95%|█████████▌| 6978/7320 [8:11:20<16:36,  2.91s/it]

6978it [8:11:20,  4.22s/it]
 95%|█████████▌| 6979/7320 [8:11:24<18:53,  3.32s/it]

6979it [8:11:24,  4.22s/it]
 95%|█████████▌| 6980/7320 [8:11:24<13:52,  2.45s/it]

6980it [8:11:24,  4.22s/it]
 95%|█████████▌| 6981/7320 [8:11:27<13:55,  2.46s/it]

6981it [8:11:27,  4.22s/it]
 95%|█████████▌| 6982/7320 [8:11:31<16:16,  2.89s/it]

6982it [8:11:31,  4.22s/it]
 95%|█████████▌| 6983/7320 [8:11:41<28:55,  5.15s/it]

6983it [8:11:41,  4.22s/it]
 95%|█████████▌| 6984/7320 [8:11:46<28:56,  5.17s/it]

6984it [8:11:46,  4.22s/it]
 95%|█████████▌| 6985/7320 [8:11:48<22:33,  4.04s/it]

6985it [8:11:48,  4.22s/it]
 95%|█████████▌| 6986/7320 [8:11:50<19:19,  3.47s/it]

6986it [8:11:50,  4.22s/it]
 95%

 97%|█████████▋| 7066/7320 [8:17:05<19:08,  4.52s/it]

7066it [8:17:05,  4.22s/it]
 97%|█████████▋| 7068/7320 [8:17:06<13:51,  3.30s/it]

7068it [8:17:06,  4.22s/it]
 97%|█████████▋| 7069/7320 [8:17:13<18:27,  4.41s/it]

7069it [8:17:13,  4.22s/it]
 97%|█████████▋| 7070/7320 [8:17:16<16:58,  4.07s/it]

7070it [8:17:16,  4.22s/it]
 97%|█████████▋| 7071/7320 [8:17:16<12:09,  2.93s/it]

7071it [8:17:16,  4.22s/it]
 97%|█████████▋| 7072/7320 [8:17:17<09:32,  2.31s/it]

7072it [8:17:17,  4.22s/it]
 97%|█████████▋| 7073/7320 [8:17:24<15:04,  3.66s/it]

7073it [8:17:24,  4.22s/it]
 97%|█████████▋| 7074/7320 [8:17:27<14:25,  3.52s/it]

7074it [8:17:27,  4.22s/it]
 97%|█████████▋| 7076/7320 [8:17:28<10:38,  2.62s/it]

7076it [8:17:28,  4.22s/it]
 97%|█████████▋| 7077/7320 [8:17:35<15:46,  3.89s/it]

7077it [8:17:35,  4.22s/it]
 97%|█████████▋| 7078/7320 [8:17:38<14:47,  3.67s/it]

7078it [8:17:38,  4.22s/it]
 97%|█████████▋| 7079/7320 [8:17:39<10:26,  2.60s/it]

7079it [8:17:38,  4.22s/it]
 97%

 98%|█████████▊| 7163/7320 [8:21:44<04:28,  1.71s/it]

7163it [8:21:44,  4.20s/it]
 98%|█████████▊| 7164/7320 [8:21:54<10:39,  4.10s/it]

7164it [8:21:54,  4.20s/it]
 98%|█████████▊| 7165/7320 [8:21:54<07:50,  3.03s/it]

7165it [8:21:54,  4.20s/it]
 98%|█████████▊| 7166/7320 [8:21:55<05:50,  2.28s/it]

7166it [8:21:55,  4.20s/it]
 98%|█████████▊| 7167/7320 [8:21:55<04:24,  1.73s/it]

7167it [8:21:55,  4.20s/it]
 98%|█████████▊| 7168/7320 [8:22:05<10:23,  4.10s/it]

7168it [8:22:05,  4.20s/it]
 98%|█████████▊| 7169/7320 [8:22:05<07:37,  3.03s/it]

7169it [8:22:05,  4.20s/it]
 98%|█████████▊| 7170/7320 [8:22:06<05:37,  2.25s/it]

7170it [8:22:06,  4.20s/it]
 98%|█████████▊| 7171/7320 [8:22:06<04:20,  1.75s/it]

7171it [8:22:06,  4.20s/it]
 98%|█████████▊| 7172/7320 [8:22:16<09:52,  4.00s/it]

7172it [8:22:15,  4.20s/it]
 98%|█████████▊| 7173/7320 [8:22:16<07:19,  2.99s/it]

7173it [8:22:16,  4.20s/it]
 98%|█████████▊| 7174/7320 [8:22:17<05:23,  2.21s/it]

7174it [8:22:17,  4.20s/it]
 98%

 99%|█████████▉| 7253/7320 [8:25:58<02:51,  2.56s/it]

7253it [8:25:58,  4.19s/it]
 99%|█████████▉| 7254/7320 [8:25:59<02:12,  2.01s/it]

7254it [8:25:59,  4.19s/it]
 99%|█████████▉| 7255/7320 [8:26:07<04:23,  4.05s/it]

7255it [8:26:07,  4.19s/it]
 99%|█████████▉| 7256/7320 [8:26:08<03:11,  3.00s/it]

7256it [8:26:08,  4.19s/it]
 99%|█████████▉| 7257/7320 [8:26:09<02:31,  2.41s/it]

7257it [8:26:09,  4.18s/it]
 99%|█████████▉| 7258/7320 [8:26:10<02:00,  1.94s/it]

7258it [8:26:10,  4.18s/it]
 99%|█████████▉| 7259/7320 [8:26:19<04:05,  4.02s/it]

7259it [8:26:19,  4.19s/it]
 99%|█████████▉| 7260/7320 [8:26:19<03:00,  3.02s/it]

7260it [8:26:19,  4.18s/it]
 99%|█████████▉| 7261/7320 [8:26:20<02:24,  2.44s/it]

7261it [8:26:20,  4.18s/it]
 99%|█████████▉| 7262/7320 [8:26:21<01:54,  1.97s/it]

7262it [8:26:21,  4.18s/it]
 99%|█████████▉| 7263/7320 [8:26:30<03:43,  3.92s/it]

7263it [8:26:30,  4.18s/it]
 99%|█████████▉| 7264/7320 [8:26:30<02:38,  2.83s/it]

7264it [8:26:30,  4.18s/it]
 99%

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,